In [1]:
import os
 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
 
API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data.pdf")
documents = loader.load()

In [3]:
documents

[Document(page_content='Frontiers in Artificial Intelligence 01 frontiersin.org\n“Better than my professor?” How \nto develop artificial intelligence \ntools for higher education\nStefano\xa0Triberti 1*, Raffaele\xa0Di\xa0Fuccio 1*, Chiara\xa0Scuotto 1,2, \nEmanuele\xa0Marsico 1 and Pierpaolo\xa0Limone 1\n1 Department of Psychology and Education, Università Telematica Pegaso, Naples, Italy, 2 Department \nof Humanistic Studies, University of Foggia, Foggia, Italy\nArtificial Intelligence (AI) tools are currently designed and tested in many fields \nto improve humans’ ability to make decisions. One of these fields is higher \neducation. For example, AI-based chatbots (“conversational pedagogical agents”) \ncould engage in conversations with students in order to provide timely feedback \nand responses to questions while the learning process is taking place and to \ncollect data to personalize the delivery of course materials. However, many \nexistent tools are able to perform tasks that 

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = FAISS.from_documents(texts, embeddings)

In [5]:
retriever = db.as_retriever()

In [6]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "search_educative_tools_creation",
    "Searches and returns excerpts from 'How to develop artificial intelligence tools for higher education'",
)
tools = [tool]

In [2]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [8]:
prompt[0].prompt.template = '''Eres un asistente al que los docentes y estudiantes de nivel secundario y universitario consultan sobre la inteligencia artifical generativa y creacion de herramientas en educacion. Para ayudar al usuario cuentas con herramientas, usalas cuando consideres apropiado. Debes tener una conversacion empatica con el usuario y responder intentando incentivar la curiosidad y el aprendizaje. Recuerda que tu objetivo es ayudar al usuario a aprender sobre la inteligencia artificial generativa y la creacion de herramientas en educacion, por lo que es deseable hacer que el usuario reflexione por ejemplo sobre como puede aplicar herramientas en su aula.'''

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.5, model="gpt-3.5-turbo-0125")

In [10]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, return_intermediate_steps=True, verbose=True)

In [11]:
agent_executor.invoke({"input": "Como han ayudado las inteligencias artificiales en la educacion en los ultimos años? (consulta la herramienta y cita la fuente)"})



> Entering new AgentExecutor chain...

Invoking: `search_educative_tools_creation` with `{'query': 'impacto de la inteligencia artificial en la educacion'}`


Triberti et al. 10.3389/frai.2024.1329605
Frontiers in Artificial Intelligence 02 frontiersin.orgThe use of AI in different contexts, such as education, has 
generated many concerns within the population. Studies show that 
although explicit attitudes are generally positive toward the use of AI, 
there is still resistance at an implicit level ( Sebri et al., 2020 ; Fietta 
et al., 2021 ; Fortuna and Gorbaniuk, 2022 ; Triberti et al., 2023 ). In 
general, people working in the numerous fields that involve (or could 
involve) AI technologies show both an explicit and implicit positive 
attitude toward AI ( Fietta et al., 2021 ; Schepman and Rodway, 2023 ). 
Regarding attitudes toward common-use tools such as ChatGPT, 
studies have shown that it is influenced not only by individual 
characteristics such as gender but also by previ

{'input': 'Como han ayudado las inteligencias artificiales en la educacion en los ultimos años? (consulta la herramienta y cita la fuente)',
 'output': 'Las inteligencias artificiales han tenido un impacto significativo en la educación en los últimos años. Según un estudio realizado por Triberti et al. (2024), se ha observado que las herramientas de inteligencia artificial, como los agentes pedagógicos conversacionales, han mejorado el proceso educativo al combinar las capacidades humanas con los recursos tecnológicos. Estos agentes pueden actuar como auxiliares en procesos de formación, gestión del conocimiento, análisis de necesidades, organización de la formación y retroalimentación de resultados.\n\nPor ejemplo, los agentes pedagógicos conversacionales tienen la capacidad de proporcionar respuestas personalizadas a las preguntas de los usuarios, estimular la reflexión personal, promover una mayor integración con el conocimiento existente y mejorar la calidad del aprendizaje a travé